In [1]:
!pip install requests BeautifulSoup4 fire

In [2]:
!pip install selenium

In [3]:
# importing required libraries
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
import string

import matplotlib.dates as mdates
import seaborn as sns

# to view all columns
pd.set_option("display.max.columns", None)

import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#sentiment analysis package
#!pip install textblob
from textblob import TextBlob

#general text pre-processor
#!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')

#tweet pre-processor 
#!pip install tweet-preprocessor
import preprocessor as p

[nltk_data] Downloading package punkt to /home/ada/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res
    
def get_tag_elements(url, tag='h2'):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    response = simple_get(url)

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        names = set()
        for li in html.select(tag):
            for name in li.text.split('\n'):
                if len(name) > 0:
                    names.add(name.strip())
        return list(names)

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url)) 
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

## 100 most influencial

In [5]:
# scrapping from the Africa Freak website
url= 'https://africafreak.com/100-most-influential-twitter-users-in-africa'
url = simple_get(url)
top_100 = get_elements(url, tag = 'h2', search  = {})
top_100[0:4]

['100. Jeffrey Gettleman (@gettleman)',
 '99. Africa24 Media (@a24media)',
 '98. Scapegoat (@andiMakinana)',
 '97. Africa Check (@AfricaCheck)']

In [6]:
#make a list of numbers from the data
splitt = [entry.split('.') for entry in top_100]
numbers = [list[0] for list in splitt]

#make a list of handles from the data
splitt = [entry.split('(') for entry in top_100]
users = [list[-1] for list in splitt]
users = [entry.split(')') for entry in users]
users = [list[0] for list in users]

#make a list of user names from the data
names = [entry.split('.') for entry in top_100]
names = [list[-1] for list in names]
names = [entry.split('(') for entry in names]
names = [list[0] for list in names]


In [7]:
print('last four ranks:', numbers[0:4])
print('last four handles:', users[0:4])
print('last four user names:', names[0:4])

last four ranks: ['100', '99', '98', '97']
last four handles: ['@gettleman', '@a24media', '@andiMakinana', '@AfricaCheck']
last four user names: [' Jeffrey Gettleman ', ' Africa24 Media ', ' Scapegoat ', ' Africa Check ']


### Creating a dataframe from the lists generated

In [8]:
data = {'users':users, 'names':names, 'ranks':numbers}
top_100 = pd.DataFrame(data)
top_100.drop(top_100.tail(5).index,inplace=True)
top_100.to_csv('top_100.csv')
top_100.tail()

,users,names,ranks
95,@Julius_S_Malema,Julius Sello Malema,5
96,@News24,News24,4
97,@SAPresident,Zuma,3
98,@GarethCliff,Gareth Cliff,2
99,@Trevornoah,Trevor Noah,1


In [247]:
# top 10 influencers
influencers = top_100.tail(10)
influencers.to_csv('top10_from_top100.csv')
influencers

,users,names,ranks
90,@Computicket,Computicket,10
91,@loyisogola,loyiso gola,9
92,@5FM,5FM,8
93,@mailandguardian,mailandguardian,7
94,@helenzille,Helen Zille,6
95,@Julius_S_Malema,Julius Sello Malema,5
96,@News24,News24,4
97,@SAPresident,Zuma,3
98,@GarethCliff,Gareth Cliff,2
99,@Trevornoah,Trevor Noah,1


## African Leaders response to covid

In [10]:
# scrapping from the Atlantic Council website
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/'
names = get_elements(url, tag ='a', search={'class':'css-4rbku5 css-18t94o4 css-1dbjc4n r-1loqt21 r-1wbh5a2 r-dnmrzs r-1ny4l3l'})
names = get_elements(url, tag ='div', search={})
names

['Issues',
 'Regions',
 'Insights & impact',
 'People',
 'Programs',
 'Events',
 '',
 'Get involved',
 'Support the Council',
 'Sign up',
 'About the Council',
 'Media',
 'Careers',
 'Home',
 'Issues',
 'Regions',
 'Search',
 'Menu',
 '+',
 'Issues',
 'Regions',
 'Insights & impact',
 'People',
 'Programs',
 'Events',
 '',
 'Get involved',
 'Support the Council',
 'Sign up',
 'About the Council',
 'Media',
 'Careers',
 '',
 'Search Suggestions',
 '{{ suggestion }}',
 '',
 'Total Results {{ currentCount }}/{{ totalCount }}',
 '{{ searchResult.post_type }}',
 '{{ searchResult.date }}',
 '{{ searchResult.author.name }}',
 'Load More',
 'Issues',
 'Region',
 'Focus',
 'Filter Results',
 'Hide',
 'Tue, Mar 24, 2020',
 'African leaders respond to coronavirus… on Twitter',
 'AfricaSource',
 'by',
 'Luke Tyburski',
 'Africa',
 'Coronavirus',
 'Media',
 'Resilience & Society',
 'Ivorian President Alassane Ouattara shows off proper sanitation practices to his Twitter following. (Twitter/@AOuatta

In [11]:
lists = []
for elements in names:
    res = re.findall(r'\(.*?\)', elements)
    lists.append(res)

handles = []
for l in lists:
    for element in l:
        if element.startswith('(@'):
            handles.append(element.strip('(,)'))
handles

['@EswatiniGovern1',
 '@MalawiGovt',
 '@hagegeingob',
 '@FinanceSC',
 '@PresidencyZA',
 '@mohzambia',
 '@edmnangagwa',
 '@MinSantedj',
 '@hawelti',
 '@StateHouseKenya',
 '@PaulKagame',
 '@M_Farmaajo',
 '@SouthSudanGov',
 '@SudanPMHamdok',
 '@TZSpokesperson',
 '@KagutaMuseveni',
 '@angola_Mirex',
 '@willynyamitwe',
 '@Cherif_MZ',
 '@Presidence_RDC',
 '@PresidentABO',
 '@PresidenceBenin',
 '@rochkaborepf',
 '@PresidenciaCV',
 '@AOuattara_PRCI',
 '@Presidency_GMB',
 '@NAkufoAddo',
 '@President_GN',
 '@USEmbalo',
 '@PresidenceMali',
 '@CheikhGhazouani',
 '@IssoufouMhm',
 '@MBuhari',
 '@Macky_Sall',
 '@PresidentBio',
 '@MSPS_Togo',
 '@EswatiniGovern1',
 '@MalawiGovt',
 '@hagegeingob',
 '@FinanceSC',
 '@PresidencyZA',
 '@mohzambia',
 '@edmnangagwa',
 '@MinSantedj',
 '@hawelti',
 '@StateHouseKenya',
 '@PaulKagame',
 '@M_Farmaajo',
 '@SouthSudanGov',
 '@SudanPMHamdok',
 '@TZSpokesperson',
 '@KagutaMuseveni',
 '@angola_Mirex',
 '@willynyamitwe',
 '@Cherif_MZ',
 '@Presidence_RDC',
 '@PresidentAB

### DataFrame from the list 

In [291]:
data = {'handles':handles}
leaders = pd.DataFrame(data)
leaders.to_csv('leaders response.csv')
print(leaders.head())
print(leaders.shape)

            handles
0  @EswatiniGovern1
1       @MalawiGovt
2      @hagegeingob
3        @FinanceSC
4     @PresidencyZA
(216, 1)


In [248]:
top10_leaders = leaders.head(10)
top10_leaders.to_csv('top10_leaders.csv')
top10_leaders

,handles
0,@EswatiniGovern1
1,@MalawiGovt
2,@hagegeingob
3,@FinanceSC
4,@PresidencyZA
5,@mohzambia
6,@edmnangagwa
7,@MinSantedj
8,@hawelti
9,@StateHouseKenya


## Twitter scrapping of the 100 influential leaders and other leaders officials.

In [215]:
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth)

df = pd.DataFrame(columns=['screen_name','description','number_of_tweets','following', 'followers',
                          'likes', 'retweets', 'hashtags', 'mentions'])
def get_it(account_list):
    for target in account_list: 
        item = auth_api.get_user(target)
        name = item.name
        screen_name = item.screen_name
        description = item.description
        number_of_tweets = item.statuses_count
        following = item.friends_count
        followers = item.followers_count
        
        # age of account
        account_created_date = item.created_at
        delta = datetime.utcnow() - account_created_date
        account_age_days = delta.days
        if account_age_days > 0:
            avg_tweets_per_day = float(number_of_tweets)/float(account_age_days)


        # tweets (hashtags and mentions)
        global hashtags, mentions, replies, comments  # making them global in order to intergrate them to the df later
        hashtags = []
        mentions = []
        comments = []
        retweet_count = []
        likes_count = []
        replies = []

        tweet_count = 0
        end_date = datetime.utcnow() - timedelta(days=180)
        for status in Cursor(auth_api.user_timeline, id=target, include_rts=False).items():                
            tweet_count += 1
            if hasattr(status, "entities"):
                entities = status.entities

    #     process_status(status)
            #hashtags
            if "hashtags" in entities:
                for ent in entities["hashtags"]:
                    if ent is not None:
                        if "text" in ent:
                            hashtag = ent["text"]
                            if hashtag is not None:
                                  hashtags.append(hashtag)

            #mentions  (will fetch other users but will later use to do mention counts between the involved users)                   
            if "user_mentions" in entities:
                for ent in entities["user_mentions"]:
                    if ent is not None:
                        if "screen_name" in ent:
                            name = ent["screen_name"]
                            if name == target:
                                if name is not None:
                                    mentions.append(name)

            # replies count                  
            if hasattr(status, "reply_count"):
                comment = status.reply_count
                if comment is not None:
                    comments.append(comment)


            # retweets count     
            if hasattr(status, "retweet_count"):
                retweets = status.retweet_count
                if retweets is not None:
                    retweet_count.append(retweets)


            # likes count     
            if hasattr(status, "favorite_count"):
                likes = status.favorite_count 
                if likes is not None:
                    likes_count.append(likes)

            # replies
            if hasattr(status, 'in_reply_to_status_id_str'):
                if (status.in_reply_to_status_id_str == status.id_str):
                    replies.append(status.text)



            # break if tweets are older than end date                 
            if status.created_at < end_date:
                break
          
            df.loc[target, ['screen_name']] = screen_name
            df.loc[target, ['description']] = description
            df.loc[target, ['number_of_tweets']] = number_of_tweets
            df.loc[target, ['following']] = following
            df.loc[target, ['followers']] = followers
            df.loc[target, ['likes']] = sum(likes_count)
            df.loc[target, ['retweets']] = sum(retweet_count)
#             df.loc[target, ['hashtags']] = hashtags
#             df.loc[target, ['mentions']] = mentions
            
#     print(hashtags)  
    return df

In [241]:
the_100 = top_100.users.unique()
the_leaders = leaders.handles.unique()
l1 = the_100.astype(str).tolist() 
l2 = the_leaders.astype(str).tolist()
accounts = l1 + l2

# take care of private, suspended or deleted accounts
names = []
for user in accounts:
    try:
        u=auth_api.get_user(user)
        names.append(u.screen_name)
    except Exception:
            pass
names     

['gettleman',
 'a24media',
 'AndiMakinana',
 'AfricaCheck',
 'JamesCopnall',
 'oafrica',
 'PatrickNgowi',
 'stateafrica',
 'Moadow',
 'BrendanSAfrica',
 'CityTshwane',
 'VISI_Mag',
 'ThisIsAfricaTIA',
 'sarzss',
 'TheEIU_Africa',
 'InvestInAfrica',
 'malonebarry',
 'ArtSouthAfrica',
 'KahnMorbee',
 'JamalMOsman',
 'iamsuede',
 'mikestopforth',
 'equal_education',
 't_mcconnell',
 'forbeesta',
 'jaxpanik',
 'ThisisAfrica',
 'audisouthafrica',
 'ONEinAfrica',
 'Hamza_Africa',
 'africatechie',
 'cx73',
 'ayittey',
 'MercedesBenz_SA',
 'africagathering',
 'OkayAfrica',
 'mary_harper',
 'savetherhino',
 'africamedia_CPJ',
 'TechCentral',
 'GautengProvince',
 'Aynte',
 'daniel_howden',
 'rangerdiaries',
 'TheStar_news',
 'schneiderhome',
 'TheAfricaReport',
 'CityofJoburgZA',
 'ThinkAfricaFeed',
 'AfricaGoodNews',
 'willintune',
 'cnbcafrica',
 'MadeItInAfrica',
 'AfricaResearch',
 'FoodBlogCT',
 'MbuyiseniNdlozi',
 'africaprogress',
 'IFCAfrica',
 'HenleyAfrica',
 'geoffreyyork',
 'Entrepre

In [259]:
data = get_it(names)
data.to_csv('twitter_data.csv')

In [297]:
# data = da.drop_duplicates(subset='screen_name')

In [299]:
data.head()

,Unnamed: 0,Unnamed: 0.1,screen_name,description,number_of_tweets,following,followers,likes,retweets,hashtags,mentions,popularity
0,0,@gettleman,gettleman,South Asia bureau chief for the New York Times...,3766,37,25701,9267,3371,NaN,NaN,12638
1,1,@a24media,a24media,Africa 24 produces compelling content that mak...,16852,3060,31294,603,194,NaN,NaN,797
2,2,@andiMakinana,AndiMakinana,In pursuit of scoops. I do not write headlines...,142183,2837,101172,73785,34232,NaN,NaN,108017
3,3,@AfricaCheck,AfricaCheck,Africa's first independent fact-checking websi...,27294,4593,68029,7209,6857,NaN,NaN,14066
4,4,@JamesCopnall,JamesCopnall,BBC reporter + presenter. Author A Poisonous T...,19434,5046,21960,908,161,NaN,NaN,1069


### Twitter handles of the 10 most influential Twitter users in Africa in order of their ***popularity*** (most influential to least influential)

Using the formulae provided in the week 1 challenge document. ***popularity_score = #retweets + #likes***

In [287]:
data['popularity'] = data['retweets'] + data['likes']
popular = data.sort_values(by = 'popularity', ascending = False)
popular.to_csv('popular_based_on_twitter.csv')
popular.head(10)

/home/ada/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,screen_name,description,number_of_tweets,following,followers,likes,retweets,hashtags,mentions,popularity
85,Trevornoah,Trevornoah,Comedian from South Africa. I was in the crowd...,11186,325,10798902,1310859,269364,NaN,NaN,1580223
82,Julius_S_Malema,Julius_S_Malema,Commander in Chief of Economic Freedom Fighter...,37178,652,3123242,1199508,254705,NaN,NaN,1454213
54,MbuyiseniNdlozi,MbuyiseniNdlozi,EFF Commissar & Member of Parliament - @EFFSou...,35887,5473,1071914,1155395,229635,NaN,NaN,1385030
118,MBuhari,MBuhari,This is the official account of Muhammadu Buha...,4730,26,3265988,1000482,278145,NaN,NaN,1278627
101,KagutaMuseveni,KagutaMuseveni,President of the Republic of Uganda,6640,28,1807416,680723,127443,NaN,NaN,808166
112,NAkufoAddo,NAkufoAddo,Official Twitter account of Nana Addo Dankwa A...,7086,351,1502265,538089,99677,NaN,NaN,637766
99,SudanPMHamdok,SudanPMHamdok,The official account of the Prime Minister of ...,649,115,368579,491083,95245,NaN,NaN,586328
90,PresidencyZA,PresidencyZA,This is the official Twitter page of The Presi...,18839,14,1595945,337064,143158,NaN,NaN,480222
20,malonebarry,malonebarry,"Executive Producer, @AJStream. Previously: Al ...",35171,2988,28423,339468,120958,NaN,NaN,460426
119,Macky_Sall,Macky_Sall,Président de la République du Sénégal 🇸🇳,2791,171,1371355,366390,57859,NaN,NaN,424249


### List the Twitter handles of the 10 most influential government officials in Africa in order of their ***influence*** (most influential to least influential)
Assuming influence is associated with reach score, using the formule ***#followers - #they follow***

In [290]:
data['influence_score'] = data['followers'] - data['following']
most_influential = data.sort_values(by = 'influence_score', ascending = False)
most_influential.to_csv('most_influential based on twitter.csv')
most_influential.head(10)

/home/ada/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,screen_name,description,number_of_tweets,following,followers,likes,retweets,hashtags,mentions,popularity,influence_score
85,Trevornoah,Trevornoah,Comedian from South Africa. I was in the crowd...,11186,325,10798902,1310859,269364,NaN,NaN,1580223,10798577
83,News24,News24,South Africa's premier online news resource. F...,322393,631,3573013,197685,81459,NaN,NaN,279144,3572382
118,MBuhari,MBuhari,This is the official account of Muhammadu Buha...,4730,26,3265988,1000482,278145,NaN,NaN,1278627,3265962
82,Julius_S_Malema,Julius_S_Malema,Commander in Chief of Economic Freedom Fighter...,37178,652,3123242,1199508,254705,NaN,NaN,1454213,3122590
96,PaulKagame,PaulKagame,"President of the Republic of Rwanda, write to:...",2861,181,1979416,194274,46499,NaN,NaN,240773,1979235
84,GarethCliff,GarethCliff,President of https://t.co/scMZ7lsVKF ⚜. Enquir...,31622,356,1974289,26361,5307,NaN,NaN,31668,1973933
101,KagutaMuseveni,KagutaMuseveni,President of the Republic of Uganda,6640,28,1807416,680723,127443,NaN,NaN,808166,1807388
73,euphonik,euphonik,LIFE | MUSIC | PROPERTY | BRANDS | CULTURE Con...,4510,65,1753245,223750,35389,NaN,NaN,259139,1753180
90,PresidencyZA,PresidencyZA,This is the official Twitter page of The Presi...,18839,14,1595945,337064,143158,NaN,NaN,480222,1595931
112,NAkufoAddo,NAkufoAddo,Official Twitter account of Nana Addo Dankwa A...,7086,351,1502265,538089,99677,NaN,NaN,637766,1501914


## Fetch Tweets of the handles in the dataframes then create csv for each account.

The run time is really long, run it only iff you want the tweets of each of the scrapped handles.
Will be used to pull the hashtags and mentions if the first method fails.

In [93]:
consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')

def get_all_tweets(screen_name):

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []  

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name, count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200, max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweepy tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]

    #write the csv  
    with open('%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)

    pass

# fetch the handles from the top_100 and leaders dataframes
# convert to list the merge them to be one list.
the_100 = top_100.users.unique()
the_leaders = leaders.handles.unique()
l1 = the_100.astype(str).tolist() 
l2 = the_leaders.astype(str).tolist()
accounts = l1 + l2

# take care of private, suspended or deleted accounts
names = []
for user in accounts:
    try:
        u=auth_api.get_user(user)
        names.append(u.screen_name)
    except Exception:
            pass
names   

if __name__ == '__main__':
    #pass in the username of the account you want to download
    for i,name in enumerate(names):
        get_all_tweets(name) 